import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter, OrderedDict

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np

In [3]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation','relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']

In [4]:
def rebin_education(education):
    if education in [' Preschool', ' 1st-4th', ' 5th-6th']:
        return 0
    elif education in [' 7th-8th', ' 9th']:
        return 1
    elif education in [' 10th', ' 11th', ' 12th']:
        return 2
    elif education in [' HS-grad', ' Some-college']:
        return 3
    elif education == ' Assoc-acdm':
        return 4
    elif education == ' Assoc-voc':
        return 5
    elif education == ' Bachelors':
        return 6
    elif education == ' Masters':
        return 7
    elif education == ' Prof-school':
        return 8
    elif education == ' Doctorate':
        return 9

In [5]:
def process_adult(df):
    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace(' ?',np.nan)
    df['workclass'] = df['workclass'].replace(' ?',np.nan)
    df['occupation'] = df['occupation'].replace(' ?',np.nan)
    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1}).astype(int)
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['workclass'] = df['workclass'].map({' Never-worked': 0, ' Without-pay': 1, ' State-gov': 2, ' Local-gov': 3, ' Federal-gov': 4, ' Self-emp-inc': 5, ' Self-emp-not-inc': 6, ' Private': 7}).astype(int)
    # df['education'] = df['education'].map({' Preschool': 0, ' 1st-4th': 1, ' 5th-6th': 2, ' 7th-8th': 3, ' 9th': 4, ' 10th': 5, ' 11th': 6, ' 12th': 7, ' HS-grad': 8, ' Some-college': 9, ' Bachelors': 10, ' Prof-school': 11, ' Assoc-acdm': 12, ' Assoc-voc': 13, ' Masters': 14, ' Doctorate': 15}).astype(int)
    df['marital'] = df['marital'].map({' Married-civ-spouse': 2, ' Divorced': 1, ' Never-married': 0, ' Separated': 1, ' Widowed': 1, ' Married-spouse-absent': 2, ' Married-AF-spouse': 2}).astype(int)
    df['relationship'] = df['relationship'].map({' Wife': 1 , ' Own-child': 0 , ' Husband': 1, ' Not-in-family': 0, ' Other-relative': 0, ' Unmarried': 0}).astype(int)
    df['race'] = df['race'].map({' White': 1, ' Asian-Pac-Islander': 0, ' Amer-Indian-Eskimo': 0, ' Other': 0, ' Black': 0}).astype(int)
    df['gender'] = df['gender'].map({' Male': 1, ' Female': 0}).astype(int)
    df['education'] = df['education'].apply(rebin_education)
    # process hours
    df.loc[(df['hours'] <= 40), 'hours'] = 0
    df.loc[(df['hours'] > 40), 'hours'] = 1
    df = df.drop(columns=['fnlwgt', 'education.num', 'occupation', 'country', 'capgain', 'caploss'])
    df = df.reset_index(drop=True)

    return df

In [6]:
df_train = pd.read_csv('adult.data', names = cols, sep=",")

In [7]:
df_train = process_adult(df_train)

In [8]:
df_train.head()

,age,workclass,education,marital,relationship,race,gender,hours,income
0,0,2,6,0,0,1,1,0,0
1,1,6,6,2,1,1,1,0,0
2,0,7,3,1,0,1,1,0,0
3,1,7,2,2,1,0,1,0,0
4,0,7,6,2,1,0,0,0,0


In [9]:
df = df_train.drop(columns='income')

In [10]:
df.head()

,age,workclass,education,marital,relationship,race,gender,hours
0,0,2,6,0,0,1,1,0
1,1,6,6,2,1,1,1,0
2,0,7,3,1,0,1,1,0
3,1,7,2,2,1,0,1,0
4,0,7,6,2,1,0,0,0


In [11]:
for i in range(5):
    print("".join(str(x) for x in df.iloc[i].values.tolist()))

02600110
16621110
07310110
17221010
07621000


In [12]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f'Unique values in column "{column}": {unique_values}')

Unique values in column "age": [0 1]
Unique values in column "workclass": [2 6 7 4 3 5 1]
Unique values in column "education": [6 3 2 7 1 4 9 5 8 0]
Unique values in column "marital": [0 2 1]
Unique values in column "relationship": [0 1]
Unique values in column "race": [1 0]
Unique values in column "gender": [1 0]
Unique values in column "hours": [0 1]


In [13]:
def match_pattern(string1, string2):
    if len(string1) == len(string2):
        for i in range(len(string1)):
            if string1[i].isdigit() and string2[i].isdigit():
                if string1[i] != string2[i]:
                    return 0
        else:
            return 1
        return 0

In [14]:

#Return the inverted index matrix for the dataset
def preprocessing(dataset):
    
    #Get cardinalities out of the dataset
    cardinalities = []
#     cardinalities = [[0,1],[0,1],[0,1]]
    for col in dataset.columns:
        cardinalities.append(dataset[col].unique().tolist())

    for cardinality in cardinalities:
        cardinality.sort()
        
    num = 1
    for cardinality in cardinalities:
        num *= len(cardinality)+1
        
    print('Total number of patterns in the dataset: ',num)


    #Get unique value combinations count
    dataset_string = []
#     dataset_string = ['010','001','000','011','001']
    for i in range(len(dataset)):
        dataset_string.append("".join(str(x) for x in dataset.iloc[i].values.tolist()))
        
    counts = OrderedDict()

    for item in dataset_string:
        counts[item] = counts.get(item, 0) + 1

    data_unique_values = list(counts.keys())
    data_value_counts = list(counts.values())
    
    inverted_ind = []
    
    #create inverted index matrix
    for i,cardinality in enumerate(cardinalities):
        for cardinality_val in cardinality:
            new_row = []
            for val in data_unique_values:
                if cardinality_val == int(val[i]):
                    new_row.append(1)
                else:
                    new_row.append(0) 
            inverted_ind.append(new_row)
            
    return inverted_ind, data_value_counts, cardinalities
        

def cov(pattern, dataset):  #tested_OK
    """
    Returns the number of instances in the dataset covered by the given pattern.
    """
    count = 0

    for i in range(len(dataset)):
        res = match_pattern(pattern, "".join(str(x) for x in df.iloc[i].values.tolist()))
        if res == 1:
            count += 1
#     print(count)
    return count


def coverage_optimized(pattern, inverted_ind, data_value_counts, cardinalities):  #invertedindices  #tested_OK for the example given in the research paper

    result_and = [1] * len(inverted_ind[0])
    row_index = 0
    for i,x in enumerate(pattern):
        if x!= 'X':
            
            #find the value x in cardinalities[i]
            index = cardinalities[i].index(int(x))
            row_index += index
            inverted_ind_row = inverted_ind[row_index]
            for j in range(len(inverted_ind_row)):
                result_and[j] = int(inverted_ind_row[j] and result_and[j])
            row_index -= index
            row_index += len(cardinalities[i])
        else:
            row_index += len(cardinalities[i])      

    # DOT Product between the above result and the count array for the datapoints
    coverage = sum([x*y for x, y in zip(data_value_counts, result_and)])
    return coverage
    

def generate_parent_nodes(pattern):  #tested_OK
    """
    Generates all parent nodes of the given pattern by replacing one deterministic
    cell with a wildcard character.
    """
    parents = []
    for i in range(len(pattern)):
        new_string = pattern[:i] + "X" + pattern[i+1:]
        if new_string != pattern:
            parents.append(new_string)
    return parents


def generate_nodes(pattern, cardinalities): #tested_OK
    """
    Generates all nodes on the given pattern and cardinalities based on Rule 1.
    """
    
    """
    TODO: Make cardinalities 2D vector, so that it can be traversed to find the children of a pattern
    """

    # Find the index of the right-most deterministic element in the pattern
    index = len(pattern) - 1
    rm_deter = -1
    while index >= 0:
        if pattern[index] != 'X':
            rm_deter = index
            break
        index -= 1

    candidate_nodes = []
    rm_deter += 1
    if rm_deter >= 0:
        while rm_deter < len(pattern):
            index = rm_deter
            for value in cardinalities[index]:
                candidate_node = pattern
                candidate_node = pattern[:index] + str(value) + pattern[index+1:] 
                candidate_nodes.append(candidate_node)
            rm_deter += 1

    return candidate_nodes


def dominance(pattern, mups):
    #iterate through the mups, find if any of the mups is an ancestor for the pattern p return 0
    # if pattern p is an ancestor for any of the mups return 1
    # else return -1
    
    for m in mups:
        for i,x in enumerate(m):
            if m[i] != pattern[i]:
                if m[i] == 'X' and pattern[i] != 'X':
                    return 0
                elif m[i] != 'X' and pattern[i] == 'X':
                    return 1
    
    return -1

    
def deepdiver(dataset, threshold):
    """
    Finds the maximal uncovered patterns in the dataset.
    """ 
    inverted_index, unique_value_counts, cardinalities = preprocessing(dataset)

    stack = ['XXXXXXXX']
    maximal_uncovered = []
    while stack:
        pattern = stack.pop()
        uncovered_flag = False
        if dominance(pattern, maximal_uncovered) == 0:
            continue
        elif dominance(pattern, maximal_uncovered) == 1:
            uncovered_flag = False
        else: 
            count = coverage_optimized(pattern, inverted_index, unique_value_counts, cardinalities)
            if count < threshold:
                uncovered_flag = True
        if uncovered_flag:
            stack0 = []
            stack0.append(pattern)
            while stack0:
                pattern0 = stack0.pop()
                parent_nodes = generate_parent_nodes(pattern0)
                for p in parent_nodes:
                    count0 = coverage_optimized(p, inverted_index, unique_value_counts, cardinalities )
                    if count0 < threshold:
                        stack0.append(p)
                        break                   
                maximal_uncovered.append(pattern)
        else:
            stack.extend(generate_nodes(pattern, cardinalities))
    
    print(len(maximal_uncovered))
    print('MUPs are: ', maximal_uncovered)
    return maximal_uncovered


In [15]:
inv, d, cardinalities = preprocessing(df)
print(coverage_optimized('XXXXXXXX',inv,d,cardinalities))


Total number of patterns in the dataset:  85536
30162


In [16]:
deepdiver(df, 100)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXX1001']


['XXXX1001']

In [17]:
deepdiver(df, 50)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXX1001']


['XXXX1001']

In [18]:
deepdiver(df, 200)

Total number of patterns in the dataset:  85536
3
MUPs are:  ['XXXX1001', 'XXXX1000', 'XXXX0001']


['XXXX1001', 'XXXX1000', 'XXXX0001']

In [19]:
deepdiver(df, 300)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXX001']


['XXXXX001']

In [20]:
deepdiver(df, 400)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXX001']


['XXXXX001']

In [21]:
deepdiver(df, 500)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXX001']


['XXXXX001']

In [22]:
deepdiver(df, 1000)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXX0X1']


['XXXXX0X1']

In [23]:
deepdiver(df, 2000)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXXX01']


['XXXXXX01']

In [24]:
deepdiver(df, 5000)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXXX01']


['XXXXXX01']

In [25]:
deepdiver(df, 8000)

Total number of patterns in the dataset:  85536
2
MUPs are:  ['XXXXXX11', 'XXXXXX01']


['XXXXXX11', 'XXXXXX01']

In [26]:
deepdiver(df, 10000)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXXXX1']


['XXXXXXX1']

In [27]:
deepdiver(df, 20000)

Total number of patterns in the dataset:  85536
1
MUPs are:  ['XXXXXXX1']


['XXXXXXX1']

In [28]:
def getStringsMatchingWithPattern(pattern, df):
    
    
    #Get unique value combinations count
    dataset_string = []
    for i in range(len(df)):
        dataset_string.append("".join(str(x) for x in df.iloc[i].values.tolist()))
        
    
    strings = []
    flag = True
    for s in dataset_string:
        flag = True
        for i,x in enumerate(s):
            if x != pattern[i]:
                if pattern[i] != 'X':
#                     print('string:',s[i])
#                     print('string:',pattern[i])
                    flag = False
                    
        if flag:
            strings.append(s)
            
    print(len(strings))       
    return strings
        
            

In [29]:
getStringsMatchingWithPattern('XXXXXXX1', df)

9197


['16321111',
 '07700101',
 '07321011',
 '07400011',
 '07121011',
 '07221111',
 '06710101',
 '07921111',
 '07310111',
 '03400111',
 '07300011',
 '07121111',
 '07621111',
 '07500111',
 '17821101',
 '14610111',
 '15310111',
 '07321111',
 '07921111',
 '07310111',
 '15321111',
 '06621111',
 '17321111',
 '07721111',
 '04900101',
 '03300111',
 '16921111',
 '07321111',
 '07621111',
 '16700101',
 '16321111',
 '17121111',
 '15321111',
 '17321111',
 '07300111',
 '07621111',
 '02621111',
 '07400101',
 '07621111',
 '06321111',
 '17321111',
 '07310111',
 '07610011',
 '04400111',
 '07600111',
 '07621111',
 '07300011',
 '04321111',
 '07721101',
 '17321111',
 '04700101',
 '17621111',
 '07310111',
 '07600011',
 '07621111',
 '17621111',
 '16321111',
 '07300111',
 '07800111',
 '07621111',
 '03721111',
 '02421111',
 '17321111',
 '07421111',
 '17510101',
 '17321111',
 '05321011',
 '07600101',
 '16721111',
 '07321101',
 '07410001',
 '17610101',
 '07821111',
 '17310111',
 '07621111',
 '03600111',
 '07300111',